In [1]:
import pandas as pd
import os
import re

In [ ]:
!ls /home/yl3427/cylab/

In [ ]:
# T14
file_dir = "/secure/shared_data/rag_tnm_results/summary/summary"
file_names = ["BRCA_T14_testing.csv","LUAD_T14_testing.csv","BRCA_T14_training.csv","LUAD_T14_training.csv"]
T14_dfs = []
for f_name in file_names:
    open_file_path = os.path.join(file_dir, f_name)
    df = pd.read_csv(open_file_path)
    df.drop(columns=['Unnamed: 0'], inplace=True)
    T14_dfs.append(df)

T14_df = pd.concat(T14_dfs, axis=0, ignore_index= True) 

In [ ]:
T14_df

In [ ]:
duplicates = T14_df.duplicated(subset='patient_filename', keep='first')
duplicates.sum()

In [ ]:
# N03
file_dir = "/secure/shared_data/rag_tnm_results/summary/summary"
file_names = ["BRCA_N03_testing.csv", "LUAD_N03_testing.csv", "BRCA_N03_training.csv", "LUAD_N03_training.csv"]
N03_dfs = []
for f_name in file_names:
    open_file_path = os.path.join(file_dir, f_name)
    df = pd.read_csv(open_file_path)
    df.drop(columns=['Unnamed: 0'], inplace=True)
    N03_dfs.append(df)

N03_df = pd.concat(N03_dfs, axis=0, ignore_index= True) 

In [ ]:
duplicates = N03_df.duplicated(subset='patient_filename', keep='first')
duplicates.sum()

In [ ]:
# merge
full_df= pd.merge(T14_df, N03_df, on=['patient_filename', 'text', 'type'], how='outer')
len(T14_df), len(N03_df), len(full_df)

In [ ]:
N03_df.head(5)

In [ ]:
full_df['n'] = full_df['n'].fillna(-1).astype('int')
full_df['t'] = full_df['t'].fillna(-1).astype('int')

In [ ]:
full_df.info()

In [ ]:
duplicates = full_df.duplicated(subset='patient_filename', keep='first')
duplicates.sum()

In [ ]:
full_df.to_csv("/secure/shared_data/rag_tnm_results/summary/merged_df.csv", index=False)

# Split into 5 folds

In [ ]:
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold
import pickle

In [ ]:
df = pd.read_csv("/secure/shared_data/rag_tnm_results/summary/merged_df.csv")

brca_t = df[df['type']=="BRCA"][["patient_filename", 't']]
brca_n = df[(df['type']=="BRCA") & (df['n']!=-1)][["patient_filename", 'n']]

luad_t = df[df['type']=="LUAD"][["patient_filename", 't']]
luad_n = df[(df['type']=="LUAD") & (df['n']!=-1)][["patient_filename", 'n']]

full_5folds_dict = defaultdict(list)

In [ ]:
skf = StratifiedKFold(n_splits=5)

# brca_t
x = brca_t["patient_filename"].values
y = brca_t["t"].values

for i, (train_index, test_index) in enumerate(skf.split(x, y)):
    each_fold = {}
    each_fold['train'] = list(x[train_index])
    each_fold['test'] = list(x[test_index])
    full_5folds_dict['brca_t_5folds'].append(each_fold)

# brca_n
x = brca_n["patient_filename"].values
y = brca_n["n"].values

for i, (train_index, test_index) in enumerate(skf.split(x, y)):
    each_fold = {}
    each_fold['train'] = list(x[train_index])
    each_fold['test'] = list(x[test_index])
    full_5folds_dict['brca_n_5folds'].append(each_fold)

# luad_t
x = luad_t["patient_filename"].values
y = luad_t["t"].values

for i, (train_index, test_index) in enumerate(skf.split(x, y)):
    each_fold = {}
    each_fold['train'] = list(x[train_index])
    each_fold['test'] = list(x[test_index])
    full_5folds_dict['luad_t_5folds'].append(each_fold)

# luad_n
x = luad_n["patient_filename"].values
y = luad_n["n"].values

for i, (train_index, test_index) in enumerate(skf.split(x, y)):
    each_fold = {}
    each_fold['train'] = list(x[train_index])
    each_fold['test'] = list(x[test_index])
    full_5folds_dict['luad_n_5folds'].append(each_fold)

In [ ]:
# saving
with open('full_5folds_dict.pkl', 'wb') as file:
    pickle.dump(full_5folds_dict, file)
    
# # loading
# with open('full_5folds_dict.pkl', 'rb') as file:
#     loaded_dict = pickle.load(file)

# embed full report&summary

In [1]:
import pandas as pd
import os
import re
from huggingface_hub import InferenceClient
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
import tiktoken
from tqdm import tqdm
import numpy
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv, find_dotenv
if not load_dotenv(find_dotenv()):
    raise Exception("Failed to load .env file")

/home/yl3427/miniconda3/envs/rag_tnm_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.getenv('OPENAI_API_KEY'),
                model_name="text-embedding-3-small"
            )
client = chromadb.PersistentClient("/home/yl3427/cylab/chroma_db")

In [3]:
client.list_collections()

[Collection(name=full_summary_emb),
 Collection(name=full_report_emb_2),
 Collection(name=full_report_emb),
 Collection(name=full_summary_emb_2)]

In [4]:
for collection in client.list_collections():
    print(collection.count())

1517
0
1517
0


In [ ]:
# for col in client.list_collections():
#     if col.name.startswith("full") or col.name.startswith("luad") or col.name.startswith("brca"):
#         client.delete_collection(col.name)

In [ ]:
# client.delete_collection("full_report_emb")
client.get_or_create_collection("full_report_emb_2")
client.get_or_create_collection("full_summary_emb_2")
client.list_collections()

여기까지 하고 summary 생성하고 와

In [9]:
summary_df = pd.read_csv("/home/yl3427/cylab/yewon_data/usingExtract_fs_merged_df.csv")

summary_df.columns

Index(['patient_filename', 't', 'text', 'type', 'n', 'prompt_for_abstract',
       'prompt_for_extract', 'abstractive_summary', 'extractive_summary',
       'zs_ans_from_text_t', 'zs_ans_from_abstract_t', 'zs_ans_from_extract_t',
       'zs_ans_from_text_n', 'zs_ans_from_abstract_n', 'zs_ans_from_extract_n',
       '0_Has_Valid_Prediction_t', '0_coded_pred_t',
       '1_Has_Valid_Prediction_t', '1_coded_pred_t',
       '2_Has_Valid_Prediction_t', '2_coded_pred_t',
       '0_Has_Valid_Prediction_n', '0_coded_pred_n',
       '1_Has_Valid_Prediction_n', '1_coded_pred_n',
       '2_Has_Valid_Prediction_n', '2_coded_pred_n', 'is_goodsum_t',
       'is_goodsum_n'],
      dtype='object')

In [10]:
len(summary_df[summary_df["is_goodsum_t"]]), len(summary_df[summary_df["is_goodsum_n"]])

(1177, 1108)

In [ ]:
# df_n = pd.read_csv("/secure/shared_data/rag_tnm_results/summary/backup_5_folds_summary/OnlyCorrectSummary_N_withSummary_merged_df.csv")
# df_t = pd.read_csv("/secure/shared_data/rag_tnm_results/summary/backup_5_folds_summary/OnlyCorrectSummary_T_withSummary_merged_df.csv")
# len(df_n), len(df_t)

In [11]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def update_collection(report_df, collection):
    pbar = tqdm(total=report_df.shape[0])

    for _, report in report_df.iterrows():
        report_text = report["text"]
        meta = {}
        meta["patient_filename"] = report["patient_filename"]
        meta["cancer_type"] = report["type"]
        meta["t_label"] = report['t']
        meta["n_label"] = report['n']
        meta['is_goodsum_t'] = report['is_goodsum_t']
        meta["is_goodsum_n"] = report["is_goodsum_n"]
        meta["report_length_cl100k_base"] = num_tokens_from_string(report_text, "cl100k_base")
        meta["abstract"] = report['abstractive_summary']
        meta["extract"] = report['extractive_summary']

        collection.update(
            ids=meta["patient_filename"],
            embeddings=openai_ef([report_text])[0],
            metadatas=meta,
            documents=report_text
        )
        pbar.update(1)
    pbar.close()

def embed_reports_in_chroma(report_df, collection):
    
    pbar = tqdm(total=report_df.shape[0])

    for _, report in report_df.iterrows():
        report_text = report["text"]
        meta = {}
        meta["patient_filename"] = report["patient_filename"]
        meta["cancer_type"] = report["type"]
        meta["t_label"] = report['t']
        meta["n_label"] = report['n']
        meta['is_goodsum_t'] = report['is_goodsum_t']
        meta["is_goodsum_n"] = report["is_goodsum_n"]
        meta["report_length_cl100k_base"] = num_tokens_from_string(report_text, "cl100k_base")

        collection.add(
            embeddings=openai_ef([report_text])[0],
            metadatas=meta,
            documents=report_text,
            ids=meta["patient_filename"]
        )
        pbar.update(1)
    pbar.close()

def embed_summary_in_chroma(report_df, collection):
    
    pbar = tqdm(total=report_df.shape[0])

    for _, report in report_df.iterrows():
        report_text = report["extractive_summary"]
        meta = {}
        meta["patient_filename"] = report["patient_filename"]
        meta["cancer_type"] = report["type"]
        meta["t_label"] = report['t']
        meta["n_label"] = report['n']
        meta['is_goodsum_t'] = report['is_goodsum_t']
        meta["is_goodsum_n"] = report["is_goodsum_n"]
        meta["report_length_cl100k_base"] = num_tokens_from_string(report_text, "cl100k_base")

        collection.add(
            embeddings=openai_ef([report_text])[0],
            metadatas=meta,
            documents=report_text,
            ids=meta["patient_filename"]
        )
        pbar.update(1)
    pbar.close()

In [14]:
full_report_collection = client.get_collection("")
# embed_reports_in_chroma(summary_df, full_report_collection)
update_collection(summary_df, full_report_collection)

100%|██████████| 1517/1517 [05:00<00:00,  5.05it/s]


In [12]:
full_summary_collection2 = client.get_collection("full_summary_emb_2")
embed_summary_in_chroma(summary_df, full_summary_collection2)

100%|██████████| 1517/1517 [04:41<00:00,  5.39it/s]


In [18]:
result = full_report_collection.get(include = ["metadatas"])
result['metadatas']

[{'abstract': 'The patient has a non-mucinous, papillary adenocarcinoma in the right lower lobe of the lung, measuring 4.5 cm with poorly differentiated areas, involving the pleura and blood vessels, and with hilar lymph node metastases. The tumor is classified as stage IIIA (pT2, pN2, pMx) according to the TNM staging system.',
  'cancer_type': 'LUAD',
  'extract': 'The patient has a peripheral, non-mucinous, papillary adenocarcinoma in the right lower lobe of the lung, measuring 4.5 cm with foci of poor differentiation, pleural infiltration, and invasion of blood vessels. There are hilar lymph node metastases present. The tumor classification is M-8140/3, G3, pT2, pV0, pN2, pMx, and the stage is IIIA. The resection is R0.',
  'is_goodsum_n': True,
  'is_goodsum_t': True,
  'n_label': 2,
  'patient_filename': 'TCGA-05-4244.3a844132-f813-4d8e-8f7d-dbae0b23d7fd',
  'report_length_cl100k_base': 109,
  't_label': 1},
 {'abstract': 'The patient has a poorly differentiated, non-mucinous pul

In [14]:
result = full_summary_collection2.get(include = ["metadatas"], where={'n_label':{'$in': [0, 1, 2, 3]}})
len(result['ids'])

1277

In [ ]:
df[(df['type']=="LUAD") & (df['n']!=-1)]

In [ ]:
df[(df['type']=="LUAD") & (df['is_goodsum_n']) & (df['n']==3)]

In [ ]:
df = pd.read_csv("/secure/shared_data/rag_tnm_results/summary/5_folds_summary/Updated_withSummary_merged_df.csv")
for mode, folds in loaded_dict.items():
    for i, fold in enumerate(folds):
        print(f"{mode} {i+1}st fold")
        label = mode.split("_")[1]
        print(label)
        for j in [0, 1, 2, 3]:
            print(f"\tlable{j}: ", len(df[(df['patient_filename'].isin(fold['train']))&(df[f'is_goodsum_{label}'])&(df[label]==j)]))
        print()

In [ ]:
len(fold['train'])

In [ ]:
summary_collection.get(where={"$and":[{"patient_filename": {'$in': fold['train']}},
                                      {f'is_goodsum_t':{"$eq":True}},
                                      {f't_label':{"$eq":3}}]})

In [19]:
emb1 = full_summary_collection.peek()['embeddings'][0]

In [25]:
result = full_summary_collection.query(query_embeddings=emb1, include = ["metadatas", "documents"], n_results=3)

result['metadatas'][0][0]

{'cancer_type': 'LUAD',
 'is_goodsum_n': True,
 'is_goodsum_t': True,
 'n_label': 2,
 'patient_filename': 'TCGA-05-4244.3a844132-f813-4d8e-8f7d-dbae0b23d7fd',
 'report_length_cl100k_base': 82,
 't_label': 1}

In [ ]:
summary_df["is_goodsum_n"]